# Federated Learning with PySyft and PyTorch - SMS spam detection

This notebook is an example of usage of [PySyft](https://github.com/OpenMined/PySyft), an open-source library built for Federate Learning and Privacy Preserving. PySyft allows its users to perform private and secure Deep Learning. It is built as an extension of some DL libraries, such as PyTorch, Keras and Tensorflow.

In this project I will show how we can use Federated Learning with the PyTorch extension of PySyft for a classification task with a simple 1-layer GRU. 

The data used for this project was the [SMS Spam Collection Data Set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) available on the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). The dataset consists of c. 5500 SMS messages, of which around 13% are spam messages.

The objective here is to simulate two remote machines (that we will call Bob and Anne), where each machine have a similar number of labeled datapoints (SMS labeled as spam or not). 

## Useful imports

In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score

import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

import warnings

warnings.filterwarnings("ignore")

## Loading data

As we are most interested in the usage of PySyft and Federated Learning, I will skip the text-preprocessing part of the project. If you are interested in how I performed the preprocessing of the raw dataset you can take a look on the script [preprocess.py](https://github.com/andrelmfarias/Private-AI/blob/master/Federated_Learning/preprocess.py) available on my [Github page](https://github.com/andrelmfarias/Private-AI).

Each data point of the `inputs.npy` dataset correspond to an array of 30 tokens obtained form each message (padded at left or truncated at right)

The `label.npy` dataset has the following unique values: `1` for `spam` and `0` for `non-spam`

In [2]:
inputs = np.load('inputs.npy')
labels = np.load('labels.npy')

In [3]:
VOCAB_SIZE = int(inputs.max()) + 1

## Training model with Federated learning

### Training and model hyperparameters

In [4]:
# Training params
EPOCHS = 15
CLIP = 5 # gradient clipping - to avoid gradient explosion (frequent in RNNs)
lr = 0.1
BATCH_SIZE = 32

# Model params
EMBEDDING_DIM = 50
HIDDEN_DIM = 10
DROPOUT = 0.2

### Initiating virtual workers with Pysyft

In this part we are going to separate the dataset in training and test sets following the ratio 80/20. Each of these datasets will be split in two and will be sent to "Bob's" and "Anne's" machines in order to **simulate remote and private data**.

Pleae note that in a real case, such datasets will be already in the remote machines and the preprocessing will be performed before hand by their own devices.

In [5]:
import syft as sy

In [6]:
labels = torch.tensor(labels)
inputs = torch.tensor(inputs)

# splitting training and test data
pct_test = 0.2

train_labels = labels[:-int(len(labels)*pct_test)]
train_inputs = inputs[:-int(len(labels)*pct_test)]

test_labels = labels[-int(len(labels)*pct_test):]
test_inputs = inputs[-int(len(labels)*pct_test):]

In [7]:
# Hook that extends the Pytorch library to enable all computations with pointers of tensors sent to other workers
hook = sy.TorchHook(torch)

# Creating 2 virtual workers
bob = sy.VirtualWorker(hook, id="bob")
anne = sy.VirtualWorker(hook, id="anne")

# threshold indexes for dataset split (one half for Bob, other half for Anne)
train_idx = int(len(train_labels)/2)
test_idx = int(len(test_labels)/2)

# Sending toy datasets to virtual workers
bob_train_dataset = sy.BaseDataset(train_inputs[:train_idx], train_labels[:train_idx]).send(bob)
anne_train_dataset = sy.BaseDataset(train_inputs[train_idx:], train_labels[train_idx:]).send(anne)
bob_test_dataset = sy.BaseDataset(test_inputs[:test_idx], test_labels[:test_idx]).send(bob)
anne_test_dataset = sy.BaseDataset(test_inputs[test_idx:], test_labels[test_idx:]).send(anne)

# Creating federated datasets, an extension of Pytorch TensorDataset class
federated_train_dataset = sy.FederatedDataset([bob_train_dataset, anne_train_dataset])
federated_test_dataset = sy.FederatedDataset([bob_test_dataset, anne_test_dataset])

# Creating federated dataloaders, an extension of Pytorch DataLoader class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, shuffle=False, batch_size=BATCH_SIZE)

### Creating simple GRU (1-layer) model with sigmoid activation for classification task

Unfortunatelly, the current version of PySyft does not support the RNNs modules of PyTorch yet. However, I was able to handcraft a simple GRU network with linear layers for this project. 

As the focus of this notebook is the usage of PySyft, I will skip the construction of the model. If you are interested in how I built the model, you can check it out on [handcrafted_GRU.py](https://github.com/andrelmfarias/Private-AI/blob/master/Federated_Learning/handcrafted_GRU.py).

In [8]:
from handcrafted_GRU import GRU

In [9]:
# Initiating the model
model = GRU(vocab_size=VOCAB_SIZE, hidden_dim=HIDDEN_DIM, embedding_dim=EMBEDDING_DIM, dropout=DROPOUT)

### Training

For now, PySyft does not support optimizers with momentum. Therefore, we are going to stick with the classical [Stochastic Gradient Descent](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) optimizer.

As our task consists of a binary classification, we are going to use the [Binary Cross Entropy Loss](https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss).

In [10]:
# Defining loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

For each epoch we are going to compute the training and validations losses, as well as the [Area Under the ROC Curve](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics) score due to the fact that the target dataset is unbalaced (only 13% of labels are positive).

In [11]:
for e in range(EPOCHS):
    
    ######### Training ##########
    
    losses = []
    # Batch loop
    for inputs, labels in federated_train_loader:
        # Location of current batch
        worker = inputs.location
        # Initialize hidden state and send it to worker
        h = torch.Tensor(np.zeros((BATCH_SIZE, HIDDEN_DIM))).send(worker)
        # Send model to current worker
        model.send(worker)
        # Setting accumulated gradients to zero before backward step
        optimizer.zero_grad()
        # Output from the model
        output, _ = model(inputs.to(torch.long), h)
        # print(f"Output: {output}\n")
        # Calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        loss_get = loss.get()
        print(f"Loss Gradient: {loss_get}")
        # Clipping the gradient to avoid explosion
        # nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        # Backpropagation step
        optimizer.step() 
        # Get the model back to the local worker
        model.get()
        losses.append(loss_get)
    
    ######## Evaluation ##########
    
    # Model in evaluation mode
    model.eval()

    with torch.no_grad():
        test_preds = []
        test_labels_list = []
        eval_losses = []

        for inputs, labels in federated_test_loader:
            # get current location
            worker = inputs.location
            # Initialize hidden state and send it to worker
            h = torch.Tensor(np.zeros((BATCH_SIZE, HIDDEN_DIM))).send(worker)    
            # Send model to worker
            model.send(worker)
            
            output, _ = model(inputs.to(torch.long), h)
            print(f"Output Eval: {output}")
            loss = criterion(output.squeeze(), labels.float())
            eval_losses.append(loss.get())
            preds = output.squeeze().get()
            test_preds += list(preds.numpy())
            test_labels_list += list(labels.get().numpy().astype(int))
            # Get the model back to the local worker
            model.get()
        
        score = roc_auc_score(test_labels_list, test_preds)
    
    print("Epoch {}/{}...  \
    AUC: {:.3%}...  \
    Training loss: {:.5f}...  \
    Validation loss: {:.5f}".format(e+1, EPOCHS, score, sum(losses)/len(losses), sum(eval_losses)/len(eval_losses)))
    
    model.train()

Loss Gradient: 0.7117383480072021
Loss Gradient: 0.7435958385467529
Loss Gradient: 0.7143404483795166
Loss Gradient: 0.7053065299987793
Loss Gradient: 0.6497811675071716
Loss Gradient: 0.701545238494873


KeyboardInterrupt: 

## Conclusion

We can see that with the PySyft library and its PyTorch extension, we can perform operations with tensor pointers such as we can do with PyTorch API (but for some limitations that are still to be addressed). 

Thanks to this, we were able to train spam detector model without having any access to the remote and private data: for each batch we sent the model to the current remote worker and got it back to the local machine before sending it to the worker of the next batch.

We can also notice that this federated training did not harm the performance of the model as both losses reduced at each epoch as expected and the final AUC score on the test data was above 97.5%.

There is however one limitation of this method: by getting the model back we can still have access to some private information. 
Let's say Bob had only one SMS on his machine. When we get the model back, we can just check which embeddings of the model changed and we will know which were the tokens (words) of the SMS.

In order to address this issue, there are two solutions: Differential Privacy and Secured Multi-Party Computation (SMPC). Differential Privacy would be used to make sure the model does not give access to some private information. SMPC, which is one kind of Encrypted Computation, in return allows you to send the model privately so that the remote workers which have the data cannot see the weights you are using.

I will show how can we perform these techniques with PySyft in a next tutorial.

Et voilà!